In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Dikson,RU,73.5069,80.5464,37.31,99,100,12.08,overcast clouds,2021-07-28 03:28:56
1,1,Gornopravdinsk,RU,60.0500,69.9000,57.13,54,10,5.35,clear sky,2021-07-28 03:30:57
2,2,Cape Town,ZA,-33.9258,18.4232,50.41,86,20,2.30,few clouds,2021-07-28 03:27:50
3,3,Hasaki,JP,35.7333,140.8333,91.29,79,100,21.63,overcast clouds,2021-07-28 03:28:53
4,4,Bredasdorp,ZA,-34.5322,20.0403,52.92,66,100,8.05,overcast clouds,2021-07-28 03:30:58


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
#locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
#max_temp = city_data_df["Max Temp"]
#temps = []
#for temp in max_temp:
#    temps.append(max(temp, 0))
    
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

preferred_city_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
3,3,Hasaki,JP,35.7333,140.8333,91.29,79,100,21.63,overcast clouds,2021-07-28 03:28:53
13,13,Rikitea,PF,-23.1203,-134.9692,74.52,79,100,14.63,overcast clouds,2021-07-28 03:31:02
15,15,Victoria,HK,22.2855,114.1577,92.03,79,99,7.00,overcast clouds,2021-07-28 03:27:51
17,17,Bethel,US,41.3712,-73.4140,74.77,87,90,1.01,overcast clouds,2021-07-28 03:31:03
18,18,Touros,BR,-5.1989,-35.4608,74.30,89,12,11.18,few clouds,2021-07-28 03:30:36
...,...,...,...,...,...,...,...,...,...,...,...
704,704,Tessalit,ML,20.1986,1.0114,85.75,46,75,11.72,broken clouds,2021-07-28 03:34:54
706,706,Kizukuri,JP,40.8061,140.3861,74.25,92,99,18.90,overcast clouds,2021-07-28 03:34:54
707,707,Tiarei,PF,-17.5333,-149.3333,78.82,69,64,6.91,broken clouds,2021-07-28 03:34:55
709,709,Birjand,IR,32.8663,59.2211,75.67,14,0,11.50,haze,2021-07-28 03:34:56


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.notnull().sum()

City_ID                373
City                   373
Country                373
Lat                    373
Lng                    373
Max Temp               373
Humidity               373
Cloudiness             373
Wind Speed             373
Current Description    373
Date                   373
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Not necessary as no n/a within data

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hasaki,JP,91.29,overcast clouds,35.7333,140.8333,
13,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,
15,Victoria,HK,92.03,overcast clouds,22.2855,114.1577,
17,Bethel,US,74.77,overcast clouds,41.3712,-73.4140,
18,Touros,BR,74.30,few clouds,-5.1989,-35.4608,
19,Kapaa,US,82.72,light rain,22.0752,-159.3190,
21,Horta,PT,71.20,broken clouds,38.5333,-28.6333,
22,Biak,ID,83.37,broken clouds,-0.9131,122.8766,
23,Kahului,US,81.66,scattered clouds,20.8947,-156.4700,
24,Kloulklubed,PW,81.10,overcast clouds,7.0419,134.2556,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hasaki,JP,91.29,overcast clouds,35.7333,140.8333,Kamisu
13,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,Rikitea
15,Victoria,HK,92.03,overcast clouds,22.2855,114.1577,Hong Kong
17,Bethel,US,74.77,overcast clouds,41.3712,-73.4140,Danbury
18,Touros,BR,74.30,few clouds,-5.1989,-35.4608,Touros
19,Kapaa,US,82.72,light rain,22.0752,-159.3190,Kapaʻa
21,Horta,PT,71.20,broken clouds,38.5333,-28.6333,Horta
22,Biak,ID,83.37,broken clouds,-0.9131,122.8766,CV. Karya Alam Perkasa
23,Kahului,US,81.66,scattered clouds,20.8947,-156.4700,Kahului
24,Kloulklubed,PW,81.10,overcast clouds,7.0419,134.2556,Klouklubed


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.notnull().sum()


City                   373
Country                373
Max Temp               373
Current Description    373
Lat                    373
Lng                    373
Hotel Name             373
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Completed by NM